In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
words = open('names.txt','r').read().splitlines()


In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)

In [4]:
import random 
random.seed(42)
random.shuffle(words)
# build the dataset
block_size = 8 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 8]) torch.Size([182625])
torch.Size([22655, 8]) torch.Size([22655])
torch.Size([22866, 8]) torch.Size([22866])


In [5]:
for x,y in zip(Xtr[:20], Ytr[:20]):
  print(''.join(itos[ix.item()] for ix in x), '-->', itos[y.item()])

........ --> y
.......y --> u
......yu --> h
.....yuh --> e
....yuhe --> n
...yuhen --> g
..yuheng --> .
........ --> d
.......d --> i
......di --> o
.....dio --> n
....dion --> d
...diond --> r
..diondr --> e
.diondre --> .
........ --> x
.......x --> a
......xa --> v
.....xav --> i
....xavi --> e


In [ ]:
#Create linear
class Linear:
    def __init__(self,fan_in,fan_out,bias=True):
        self.weight = torch.randn(fan_in, fan_out)/ fan_out * 0.5
        self.bias = torch.zeros(fan_out) if bias else None
    def __call__(self,x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out
    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])
    

#Create BachNorm1D
class BatchNorm1D:
    def __init__(self, dim,eps= 1e-5,momentum=0.1):
        self.momentum = momentum
        self.eps = eps
        self.training = True
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)
    def __call__(self,x):
        if self.training:
            if x.ndim == 2:
                dim = 0
            elif x.ndim == 3:
                dim = (0,1)
            xmean = x.mean(0,keepdim=True)
            xvar = x.var(0,keepdim=True)
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        #buffer
        if self.training:
            with torch.no_grad():
                self.running_mean = (1-self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1-self.momentum) * self.running_var + self.momentum * xvar
        return self.out
    def parameters(self):
        return [self.beta,self.gamma] # for backpropagation


class Tanh:
  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out
  def parameters(self):
    return []
  

class FlattenConsecutive:
    def __init__(self,n):
        self.n = n
    def __call__(self,x):
        B,T,C = x.shape
        x = x.view(B,T//self.n,C*self.n)
        if x.shape[1] == 1:
            x = x.squeeze(1)
        self.out = x
        return self.out
    def parameters(self):
        return []
    
class Embedding:
  
    def __init__(self, num_embd, embd_dim):
        self.weight = torch.randn((num_embd, embd_dim))
    
    def __call__(self, IX):
        self.out = self.weight[IX]
        return self.out
  
    def parameters(self):
        return [self.weight]
    

class Sequential:
    def __init__(self, layers):
        self.layers = layers
    def __call__(self, x):
        for layer in self.layers:
             x = layer(x)
        self.out = x
        return self.out
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters() ]
                


class GLU:
    def __init__(self,chan_in,chan_out,bias=True):
        self.weight = torch.randn(chan_in,chan_out)/(chan_in)*0.5
        self.bias = torch.zeros(chan_out) if bias else None
    def __call__(self,x):
        assert x.ndim >=2, "Input tensor must have at least 2 dimensions"
        linear = x @ self.weight
        if self.bias is not None:
            linear += self.bias
        self.out = torch.tanh(linear) * (1/1 + torch.exp(-(self.weight)))
        return self.out
    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])
    




class Softmax:
    def __call__(self,x):
        x = torch.max(x, dim=-1, keepdim=True)
        z = torch.exp(x)
        sum_z = z.sum()
        self.out = z/sum_z
        return self.out

In [7]:
torch.manual_seed(42); #set rng for reproductivity

In [14]:
n_emb = 24 #dimensionality of the character embedding vectors
n_hidden = 128 #Number of neurons in the hidden layer
# C = torch.randn((vocab_size,n_emb))
# model = Sequential([
#   Embedding(vocab_size, n_emb),
#   FlattenConsecutive(2), Linear(n_emb * 2, n_hidden, bias=False), BatchNorm1D(n_hidden), Tanh(),
#   FlattenConsecutive(2), Linear(n_hidden*2, n_hidden, bias=False), BatchNorm1D(n_hidden), Tanh(),
#   FlattenConsecutive(2), Linear(n_hidden*2, n_hidden, bias=False), BatchNorm1D(n_hidden), Tanh(),
#   Linear(n_hidden, vocab_size),
# ])

model = Sequential([
   Embedding(vocab_size, n_emb),
   FlattenConsecutive(2), GLU(n_emb * 2, n_hidden, bias=False), BatchNorm1D(n_hidden), Tanh(),
   FlattenConsecutive(2), GLU(n_hidden*2, n_hidden, bias=False), BatchNorm1D(n_hidden), Tanh(),
   FlattenConsecutive(2), GLU(n_hidden*2, n_hidden, bias=False), BatchNorm1D(n_hidden), Tanh(),
   GLU(n_hidden, vocab_size),
 ])


with torch.no_grad():
  model.layers[-1].weight *= 0.1

parameters = model.parameters()
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

76579


In [15]:
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
    #mini batch
    ix = torch.randint(0,Xtr.shape[0],(batch_size,)) #batch size
    Xb,Yb = Xtr[ix],Ytr[ix]
    #forward pass
    # emb = C[Xb] #embed the characters into vectors
    # x = emb.view(emb.shape[0],-1) #concatenate the vectors
    logits = model(Xb)
    loss = F.cross_entropy(logits,Yb)
    #back pass
    for p in parameters:
        p.grad = None
    loss.backward()

    #update
    # lr = lrs[i]
    lr = 0.1 if 1 < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    #track stats
    if i % 1000 == 0:
        print(f'{i:7d}/{max_steps: 7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

      0/ 200000: 3.2980
   1000/ 200000: 2.1462
   2000/ 200000: 2.1083
   3000/ 200000: 2.3964
   4000/ 200000: 2.2212
   5000/ 200000: 1.9426
   6000/ 200000: 2.0483
   7000/ 200000: 1.9543
   8000/ 200000: 2.2971
   9000/ 200000: 2.3821
  10000/ 200000: 2.2351
  11000/ 200000: 2.4654
  12000/ 200000: 1.9040
  13000/ 200000: 2.1101
  14000/ 200000: 2.1189
  15000/ 200000: 2.5206
  16000/ 200000: 2.2419
  17000/ 200000: 2.4427
  18000/ 200000: 2.2807
  19000/ 200000: 1.9829
  20000/ 200000: 2.0883
  21000/ 200000: 2.2476
  22000/ 200000: 1.9121
  23000/ 200000: 2.0251
  24000/ 200000: 2.5959
  25000/ 200000: 1.9664
  26000/ 200000: 2.0941
  27000/ 200000: 1.9138
  28000/ 200000: 2.3344
  29000/ 200000: 2.1372
  30000/ 200000: 1.7889
  31000/ 200000: 2.1765
  32000/ 200000: 1.7451
  33000/ 200000: 1.7886
  34000/ 200000: 2.2570
  35000/ 200000: 1.8566
  36000/ 200000: 1.7493
  37000/ 200000: 1.8680
  38000/ 200000: 1.8875
  39000/ 200000: 2.4067
  40000/ 200000: 2.0336
  41000/ 200000:

In [10]:
#layers in eval mode
for layer in model.layers:
  layer.training = False

In [11]:
# evaluate train and val loss
@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  logits = model(x)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 1.9798122644424438
val 2.067711591720581


In [12]:
#model test
for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass
      logits = model(torch.tensor([context]))
      # sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

lansan.
dyluna.
janella.
kashfer.
tollersa.
eleck.
lantri.
naiman.
rick.
melamii.
say.
syaraquaz.
romek.
emmerie.
ilynn.
andruelan.
juli.
kyamlew.
colin.
rubik.
